In [ ]:
import heapq,os


#Class for implementation of a Binary Tree
class BinaryTreeNode:

    def __init__(self,value,freq):
    
        self.value = value #stores the character
        self.freq = freq   #stores the character frequency
        self.left = None
        self.right = None
    
    #for comapring the frequencies of two different characters
    def __lt__(self,obj):
        
        return self.freq < obj.freq
    
    #for comapring the frequencies of two different characters
    def __eq__(self,obj):
        
        return self.freq == obj.freq    
    
    
#Huffman Encoding and Decoding
class HuffmanCoding:
    
    def __init__(self,path):
        
        self.path = path  #stores path of the text file to be enocded/decoded
        self.__heap = []  #a heap data structure for calculating the min frequency character
        self.__codes = {} #stores the codes for mapping characters to specific numbers
        self.__ReverseCodes = {} #stores the codes for mapping specific numbers to their particular characters
        
        
    # making a frequency dictionary of words
    def __make_freq_dict(self,text):
        
        freq_dict = {}
        
        for char in text:
            
            if char not in freq_dict:
                freq_dict[char] = 0
            freq_dict[char] += 1
        
        return freq_dict
    
    
    #building a min heap and pushing Binary Tree Nodes in it.
    def __build_heap(self,freq_dict):
        
        for key in freq_dict:
            
            frequency = freq_dict[key]
            
            #making a binary tree node with character value and frequency
            binary_tree_node = BinaryTreeNode(key,frequency)
            
            #pushing the node in heap
            heapq.heappush(self.__heap,binary_tree_node)
            
            
    #building a binary tree
    def __buildTree(self):
        
        while len(self.__heap) > 1 :
            
            #getting the first minimum_frequency character from heap
            binary_tree_node_1 = heapq.heappop(self.__heap) 
            
            #getting the next minimum_frequency character from heap
            binary_tree_node_2 = heapq.heappop(self.__heap) 
            
            #add the frequencies and make a newNode with that frequency
            freq_sum = binary_tree_node_1.freq + binary_tree_node_2.freq
            newNode = BinaryTreeNode(None, freq_sum)
            
            #add the new Node in the binary tree
            newNode.left = binary_tree_node_1
            newNode.right = binary_tree_node_2
            
            #push the new node in the heap
            heapq.heappush(self.__heap,newNode)
            
        return
            
        
    #helper function for building codes and reverse codes
    def __buildCodesHelper(self,root,curr_bits):
        
        if root is None:
            return
        
        if root.value is not None:
            self.__codes[root.value] = curr_bits
            self.__ReverseCodes[curr_bits] = root.value
            return
        
        #if the character is present at left we append 0 to its code
        self.__buildCodesHelper(root.left,curr_bits+"0")
        
        #if the character is present at right we append 1 to its code
        self.__buildCodesHelper(root.right,curr_bits+"1")
    
    
    #function to build both types of codes
    def __buildCodes(self):
        
        #gettiing the top element(character) of the heap
        root = heapq.heappop(self.__heap)
        
        #getting the code for that character
        self.__buildCodesHelper(root,"")
    
    
    #getting the encoded text
    def __get_encoded_text(self,text):
        
        encoded_text = ""
        
        #for each character in the original text we get its code form the dictionary
        for char in text:
            encoded_text += self.__codes[char]
            
        return encoded_text
    
    
    #padding the encoded text to ensure that its length stays in the multiples of 8 for allowing of bit-byte conversion
    def __get_padded_encoded_text(self,encoded_text):
        
        #getting the length required to pad the encoded text
        pad_len = 8 - (len(encoded_text)%8)
        
        #adding 0's at last of encoded text
        for i in range(pad_len):
            encoded_text += "0"
            
        #storing the length of padded bits in a 8 bit binary format
        padded_info = "{0:08b}".format(pad_len)
        
        #adding the padded bits' information at front of encoded text
        padded_encoded_text = padded_info + encoded_text
        
        return padded_encoded_text
    
    
    #getting the bytes array, bit to byte conversion
    def __getBytesArray(self,padded_encoded_text):
        
        array = []
        for i in range(0,len(padded_encoded_text),8):
            
            #selecting batches of length 8 from padded encoded text 
            byte = padded_encoded_text[i:i+8]
            
            #conversion into base 2 numbers
            array.append(int(byte,2))
            
        return array
    
    
    #process to compress the given file
    def compress(self):
        
        # get file from path
        file_name, file_extension = os.path.splitext(self.path)
        
        #make an output path for binary file to be returned
        output_path = file_name + ".bin"
        
        #read text from file and write to output file in binary mode
        with open(self.path,'r+') as file, open(output_path,'wb') as output:
            text = file.read()
            text = text.rstrip() #removing spaces
            
        #make frequency dictionary using the text
            freq_dict = self.__make_freq_dict(text)
        
        #construct the heap from frequency dictionary
            self.__build_heap(freq_dict)
        
        #construct binary tree from heap
            self.__buildTree()
        
        #construct the codes from binary tree
            self.__buildCodes()
        
        # create the encoded text
            encoded_text = self.__get_encoded_text(text)
        
        #pad this encoded text
            padded_encoded_text = self.__get_padded_encoded_text(encoded_text)
        
        #getting the binary array
            bytes_array = self.__getBytesArray(padded_encoded_text)
        
        #return the binary file as output
            final_bytes = bytes(bytes_array)
            output.write(final_bytes)
            
        print("Compressed")
        
        return output_path
    
    
    #Process of decompressing the file begins here 
    
    
    #remove the padding from front and end of binary array
    def __removePadding(self,bit_string):
        
        #getting the number of 0's appended at last of the encoded file
        padded_info = bit_string[0:8]
        
        #conversion into the number of 0's
        extra = int(padded_info,2)
        
        #removing the first 8 bits
        bit_string = bit_string[8:]
        
        #removing the last added 0's
        new_text = bit_string[:-1 * extra]
        
        return new_text
    
    
    #decode the encoded text into original text
    def __decodeText(self,text):
        
        decoded_text = ""
        current_bits = ""
        
        for bit in text:
            
            #checking for bits
            current_bits += bit
            
            #using reverse code to map numbers ot their characrters
            if current_bits in self.__ReverseCodes:
                char = self.__ReverseCodes[current_bits]
                decoded_text += char
                current_bits = ""
                
        return decoded_text
        
    
    #decompressiong function
    def decompress(self,input_path):
        
        #getting the path
        file_name, file_extension = os.path.splitext(self.path)
        
        #output path for decompressed file
        output_path = file_name + "_decompressed.txt"
        
        #opening the file in read binary mode and the output file in write mode
        with open(input_path,'rb') as file, open(output_path,'w') as output:
            bit_string = ""
            
            #reading 1 byte at a time
            byte = file.read(1)
            
            while byte:
                
                # return the value of the byte when the argument is an 8-bit string
                byte = ord(byte)
                
                #converting it into a binary file and adjusting the remaining length with zeros
                bits = bin(byte)[2:].rjust(8,'0')
                
                
                bit_string += bits
                
                byte = file.read(1)
        #getting the encoded text    
            actual_text = self.__removePadding(bit_string)
        
        #getting original text
            decompressed_text = self.__decodeText(actual_text)
            output.write(decompressed_text)
        return

    
    
#getting the path
path = input()

#setting up the required functions
h = HuffmanCoding(path)

#compressing the file
output_path = h.compress()

#decompressing the file
h.decompress(output_path)